In [1]:
# pip install langchain
# pip install langchain

In [2]:
# pip install langchain_community

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")#, verbose=True)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

# chain = prompt | llm
# chain.invoke({"input": "What are the advantages of Golang"})

In [5]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [6]:
chain = prompt | llm | output_parser
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| Ollama(model='llama3')
| StrOutputParser()

In [7]:
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.callbacks import StdOutCallbackHandler

chain.invoke({"input": "How can I learn Golang quickly?"}, config={'callbacks': [ConsoleCallbackHandler()]})

In [8]:

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [9]:
print(f"the type of docs is {type(docs)}, len:{len(docs)}")

the type of docs is <class 'list'>, len:1


In [10]:
docs[0]

Document(page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly und

In [11]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [13]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama3', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [14]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [15]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)


In [16]:
print(f"documents len={len(documents)}\n the first two are")
print(documents[0])
print(documents[1])

documents len=4
 the first two are
page_content='LangSmith User Guide | 🦜️🛠️ LangSmith' metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}
page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith i

In [17]:
vector = FAISS.from_documents(documents, embeddings)

In [18]:
print(vector)

In [19]:
vector.search("What is LangSmith", "mmr")

[Document(page_content='LangSmith User Guide | 🦜️🛠️ LangSmith', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}),
 Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and h

In [20]:
from langchain.chains.combine_documents import create_stuff_documents_chain
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)


In [21]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [22]:
ans = retrieval_chain.invoke({"input": "how can langsmith help with testing"})

In [23]:
print(ans['answer'])

# Conversational Retrieval Chain

In [24]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")]
)

retrieval_chain = create_history_aware_retriever(llm, retriever, prompt)


In [25]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
ans = retrieval_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how"
    }
)


In [26]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retrieval_chain, document_chain)

In [27]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
# ans = retrieval_chain.invoke({
#     "chat_history": chat_history,
#     "input": "Tell me how"
# })

In [28]:
# print(ans['answer'])

# Agent
NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

I don't have a valid OpenAI API key yet, so it doesn't work. I am going to explore "Agent" stuff in the future.

In [29]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [30]:
import os
%env TAVILY_API_KEY="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"
#api_key=os.getenv('TAVILY_API_KEY')

env: TAVILY_API_KEY="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"


In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [32]:
tools = [retriever_tool, search]

In [33]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [34]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

prompt = hub.pull("hwchase17/openai-functions-agent")

In [35]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='

In [37]:
from langchain_openai import ChatOpenAI
gpt3 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 
agent = create_openai_functions_agent(gpt3, tools, prompt) # you can't use other local LLM model to fill in this function
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [38]:
# agent_executor.invoke({
#     "input": "how can langsmith help with testing?"
# })
# agent_executor.invoke({"input": "what is the weather in SF?"})
# chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
# agent_executor.invoke({
#     "chat_history": chat_history,
#     "input": "Tell me how"
# })



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sess-zC***********************************KRJ". You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}